# Régression linéaire et moindres carrés: résumé et applications.

## 0. Remarques.

Dans ce notebook nous illustrons et complétons empiriquement un certains nombre de points du cours sans(ou en se limitant d') entrer dans les détails et précisions formelles qu'il est néanmoins nécessaire de connaître pour une pratique/application/lecture rigoureuse des méthodes étudiées. 

Cela peut concerner la discussion de certaines hypothèses utilisées, et/ou des définitions et notations données dans le cours.

## 1. Le modèle.

### 1.1 Rappels.

On cherche à mesurer l'effet d'un groupe de variables $X \in \mathcal{X}\subseteq \mathbb{R}^K$, traditionnellement appelées régresseurs, sur une autre variable $Y\in \mathcal{Y}\subseteq\mathbb{R}$. On dispose de données  sur $(Y, X)$, à savoir un échantillon de taille $n$,  $\{(Y_i, X_i)\}_{i=1}^n$, où $Y_i$ est une variable aléatoire et $X_i := (X_{i1}, \ldots, X_{iK})^\top$ est un vecteur $K\times 1$(de variables aléatoires). Ces observations seront supposées identiquement et indépendamment distribuées(i.i.d. en abrégé). 

L'outil auquel nous allons nous intéresser dans ce cours pour étudier la relation entre la variable dépendante et les régresseurs est l'espérance conditionnelle de $Y_i$ sachant $X_i$, $\mathrm{E}(Y_i| X_i)$, laquelle vue comme une fonction de $X_i$ est appelée fonction de régression(ou plus succinctement régression) de $Y_i$ sur $X_i$.

La différence entre $Y_i$ et son espérance conditionnelle est appelée terme d'erreur(ou plus succinctement erreur),
\begin{align}
U_i = Y_i - \mathrm{E}(Y_i| X_i)
\end{align} 

Nous allons à présent supposer que $\mathrm{E}(Y_i|X_i)$ est linéaire par rapport à un vecteur de paramètres inconnus,
\begin{align}
\mathrm{E}(Y_i|X_i) = X_{i1}\beta_1 +  X_{i2}\beta_2 + ...+ X_{iK}\beta_K = X_i^\top\beta
\end{align}

où $\beta$ est le vecteur $K\times 1$ des paramètres(ou coefficients) $\beta := (\beta_{1}, \ldots, \beta_{K})^\top$, du modèle. 

Notons aussi que comme
\begin{align*}
\beta_k = \frac{\partial \mathrm{E}(Y_i|X_i)}{\partial X_{ik}}, \ \ k=1,2,...,K.
\end{align*}
le vecteur $\beta$ est le vecteur des effets marginaux des régresseurs, i.e., $\beta_k$ donne la variation dans l'espérance  conditionnelle de $Y_i$ lorsque le régresseur $X_{ik}$ varie, pour des valeurs fixes des autres régresseurs $X_{il}$, $l=1,2,...,K$, $l\neq k$. Ceci est une des raisons pour lesquelles un des principaux objectifs est l'estimation du vecteur inconnu $\beta$ à partir des données.

Observons que les équations précédentes permettent d'écrire,
\begin{align}
Y_i = X_i^\top\beta + U_i
\end{align}
où par définition de $U_i$,
\begin{align}
\mathrm{E}(U_i|X_i) = 0
\end{align}

Ceci implique que les régresseurs ne contiennent aucune information quant à l'écart entre $Y_i$ et sont espérance conditionnelle. En outre, la règle des conditionnement successifs implique que les erreurs ont une espérance nulle: $\mathrm{E}(U_i) = 0$. 



### 1.2 Exemple: Card(1995)
Nous allons tout au long de ce notebook illustrer nos propos empiriquement en nous appuyant sur les données du travail de David Card(1995) : "Using Geographic Variation in College Proximity to Estimate the Return to Schooling". Pour une version de 1993 voir ici:

https://davidcard.berkeley.edu/papers/geo_var_schooling.pdf

Nous allons télécharger les données qui sont disponibles(outre le site de David Card lui même) à partir du site de Bruce Hansen. Une description du fichier est ici:

https://www.ssc.wisc.edu/~bhansen/econometrics/Card1995_description.pdf

Il est aussi conseillé de lire la présentation des données dans le travail de David Card.

In [30]:
# Lecture des données
## Importation des bibliothéques employées ci-après.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
## Données
url = "https://www.ssc.wisc.edu/~bhansen/econometrics/Card1995.dta"
df = pd.read_stata(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3613 entries, 0 to 3612
Data columns (total 52 columns):
id          3613 non-null int16
nearc2      3613 non-null int8
nearc4      3613 non-null int8
nearc4a     3613 non-null int8
nearc4b     3613 non-null int8
ed76        3613 non-null int8
ed66        3613 non-null int8
age76       3613 non-null int8
daded       3613 non-null float32
nodaded     3613 non-null int8
momed       3613 non-null float32
nomomed     3613 non-null int8
weight      3613 non-null int32
momdad14    3613 non-null int8
sinmom14    3613 non-null int8
step14      3613 non-null int8
reg661      3613 non-null int8
reg662      3613 non-null int8
reg663      3613 non-null int8
reg664      3613 non-null int8
reg665      3613 non-null int8
reg666      3613 non-null int8
reg667      3613 non-null int8
reg668      3613 non-null int8
reg669      3613 non-null int8
south66     3613 non-null int8
work76      3613 non-null int8
work78      3613 non-null int8
lwage76     3010 

In [37]:
# Echantillon de travail
## Nous ne retenons que les variables qui sont utilisées dans les estimations 
## de la colonne 2 du tableau 2 dans l'article, et les observations sans valeurs manquantes pour ces variables.
card = df[["lwage76", "ed76", "age76", "south66", "black", "smsa76r", "smsa66r"]]
card = card.dropna()
## On convertit les valeurs des variables en réels("float numbers")
for col in list(card):
    card[col] = card[col].astype(float)
card.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3010 entries, 0 to 3612
Data columns (total 7 columns):
lwage76    3010 non-null float32
ed76       3010 non-null int8
age76      3010 non-null int8
south66    3010 non-null int8
black      3010 non-null int8
smsa76r    3010 non-null int8
smsa66r    3010 non-null int8
dtypes: float32(1), int8(6)
memory usage: 52.9 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3010 entries, 0 to 3612
Data columns (total 7 columns):
lwage76    3010 non-null float64
ed76       3010 non-null float64
age76      3010 non-null float64
south66    3010 non-null float64
black      3010 non-null float64
smsa76r    3010 non-null float64
smsa66r    3010 non-null float64
dtypes: float64(7)
memory usage: 188.1 KB


In [42]:
# Création de certaines variables.
## Mesure de l'expérience('exp'), de l'expérience au carré('expsq'), 
## et de l'expérience au carré divisée par 100('expsq').
card['exp'] = card.age76 - card.ed76 + 6 
card['expsq'] = card['exp']**2
card['expsq100'] = card['expsq']/100
card.head()

,lwage76,ed76,age76,south66,black,smsa76r,smsa66r,exp,expsq,expsq100
0,6.306275,7.0,29.0,0.0,1.0,1.0,1.0,28.0,784.0,7.84
1,6.175867,12.0,27.0,0.0,0.0,1.0,1.0,21.0,441.0,4.41
2,6.580639,12.0,34.0,0.0,0.0,1.0,1.0,28.0,784.0,7.84
3,5.521461,11.0,27.0,0.0,0.0,1.0,1.0,22.0,484.0,4.84
4,6.591674,12.0,34.0,0.0,0.0,1.0,1.0,28.0,784.0,7.84


In [ ]:
# Régresseurs et variable dépendante.
X  = card[['ed76', 'exp', 'expsq100', 'black', 'south66', 'smsa76r']]
## On ajoute un régresseur contant.
X['const'] = 1
Y = card['lwage76']

L'équation du modèle dont nous allons estimer les paramètres est donc pour cette application:
$$
lwage_i = \beta_0 + \beta_1ed76 + \beta_2exp + \beta_3expsq100 + \beta_4 black + \beta_5 south66 + \beta_6smsa76r + U_i, \ \ i= 1,\ldots, n
$$

et pour laquelle en suppose afin de pouvoir faire une interprétation causale des paramètres que $\mathrm{E}(U_i| X_i) = 0$(où donc $Y_i := lwage_i$, $X_i :=(1, ed76_i, exp, expsq100_i, black_i, south66_i, smsa76r_i)^\top$).


# Estimation par moindres carrés ordinaires.

Nous allons utiliser l'estimateur des moindres carrés ordinaires pour estimer les paramètres du modèle. Nous savons(voir notes de cours) que cette estimateur peut être obtenu à partir des hypothèses du modèle(en particulier que $\mathrm{E}(U|X_i) = 0$) et application de la méthode des moments, et que cet estimateur est celui des moindres carrés ordinaires dont le nom résulte de qu'il peut (aussi) être défini comme solution de:

$$
\min_{(\beta_0, \beta_1, \beta_2, \beta_3, \beta_4, \beta_5, \beta_6)}n^{-1}\sum_{i=1}^n\left(lwage_i - (\beta_0 + \beta_1ed76 + \beta_2exp + \beta_3expsq100 + \beta_4 black + \beta_5 south66 + \beta_6smsa76r)\right)^2
$$

## Remarque: 
Pour faire cette estimation(comme pour beaucoup d'autres choses) avec Python il existe 
différentes façon de procéder allant de programmer les estimateurs avec les formules matricielles à des fonction disponibles dans des bibliothèques existantes. Ici nous allons utiliser la fonction "OLS" dans "statsmodels".

In [50]:
import statsmodels.api as sm 
long_reg=sm.OLS(Y,sm.add_constant(X), missing = 'drop')).fit(cov_type='HC0')
# Remarque:
# Si nous n'avions pas inclus une constante dans X nous aurions pu utiliser la syntaxe équivalente:
# long_reg=sm.OLS(Y,sm.add_constant(X)).fit(cov_type='HC0')
# Une autre syntaxe(il en existe d'autres) est aussi:
# long_reg=sm.OLS(endog = card['lwage'], exog = card[['ed76', 'exp', 'expsq100', 'black', 'south66', 'smsa76r']]).fit(cov_type='HC0')
print(long_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                lwage76   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     208.8
Date:                Thu, 08 Oct 2020   Prob (F-statistic):          4.45e-223
Time:                        20:04:29   Log-Likelihood:                -1323.3
No. Observations:                3010   AIC:                             2661.
Df Residuals:                    3003   BIC:                             2703.
Df Model:                           6                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.3678      0.172     19.587      0.0